In [5]:
import pandas as pd
data = pd.read_parquet('df_modelo2.parquet')


In [6]:
data

,business_id,business_name,city,latitude,longitude,business_rating,review_text,review_stars,review_date
0,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,The New Chipotle Mexican Grill conveniently lo...,4,2017-08-30
1,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,I am a chipotle enthusiast but from the first ...,5,2018-01-18
2,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,Went into this location about an hour before t...,5,2018-09-08
3,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,I come to this chipotle often and it's one of ...,5,2019-06-24
4,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,"The food is good, as per usual at any Chipotle...",2,2018-06-20
...,...,...,...,...,...,...,...,...,...
85608,None,Taco Bus Miami,Miami,NaN,NaN,4.4,(Translated by Google) The best taco bus foodt...,5,2021-03-13
85609,None,Taco Bus Miami,Miami,NaN,NaN,4.4,(Translated by Google) Very rich everything\n\...,5,2021-02-23
85610,None,Taco Bus Miami,Miami,NaN,NaN,4.4,(Translated by Google) LITTLE PADDING IN THE T...,3,2021-03-03
85611,None,Taco Bus Miami,Miami,NaN,NaN,4.4,(Translated by Google) Excellent food in the b...,5,2020-09-11


In [7]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

# Cargar los datos desde el archivo parquet
data = pd.read_parquet('df_modelo2.parquet')

# Convertir la columna review_date a formato numérico
data['review_date'] = pd.to_datetime(data['review_date']).dt.strftime('%Y%m%d').astype(int)

# Filtrar los datos para los restaurantes Taco Bell
taco_bell_data = data[data['business_name'].str.contains('Taco Bell', case=False)]

# Seleccionar las características y la variable objetivo
features = ['review_date', 'city']
target = 'business_rating'

# Codificar la columna 'city' utilizando one-hot encoding
encoder = OneHotEncoder(handle_unknown='ignore')
city_encoded = encoder.fit_transform(taco_bell_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
city_encoded_df = pd.DataFrame(city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con las otras características
taco_bell_data_encoded = pd.concat([taco_bell_data[['review_date']].reset_index(drop=True), city_encoded_df], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(taco_bell_data_encoded, taco_bell_data[target], test_size=0.2, random_state=42)

# Crear el modelo de regresión XGBoost
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular la métrica R^2 (coeficiente de determinación)
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)


Mean Squared Error: 0.07527204802418566
R^2 Score: 0.8546810257840202


In [3]:
# Ejemplo de uso: predecir el business_rating para Taco Bell en los próximos años
new_data = pd.DataFrame({'review_date': [20230101, 20240101, 20250101], 'city': ['Miami', 'Orlando', 'Tampa']})

# Codificar la columna 'city' del nuevo conjunto de datos utilizando el mismo encoder
new_city_encoded = encoder.transform(new_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
new_city_encoded_df = pd.DataFrame(new_city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con la columna 'review_date'
new_data_encoded = pd.concat([new_data[['review_date']], new_city_encoded_df], axis=1)

# Asegurarse de que las columnas del nuevo conjunto de datos coincidan con las del conjunto de entrenamiento
missing_cols = set(X_train.columns) - set(new_data_encoded.columns)
for c in missing_cols:
    new_data_encoded[c] = 0
new_data_encoded = new_data_encoded[X_train.columns]

# Realizar predicciones con el modelo
new_predictions = model.predict(new_data_encoded)

print("Predicted business_rating for Taco Bell in the next years:")
for year, prediction in zip([2023, 2024, 2025], new_predictions):
    print(f"Year {year}: {prediction}")

Predicted business_rating for Taco Bell in the next years:
Year 2023: 3.6241719722747803
Year 2024: 3.6487228870391846
Year 2025: 1.5888773202896118


In [ ]:
# Graficar los valores predichos vs. los valores reales
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Valores Reales')
plt.plot(y_pred, label='Valores Predichos')
plt.legend()
plt.xlabel('Índice')
plt.ylabel('Business Rating')
plt.title('Comparación de Valores Predichos vs. Valores Reales')
plt.show()